## Projeto 1 - 

In [11]:
# !pip install --upgrade pip

In [1]:
# !pip install pandas
# # !pip install csv
# !pip install git+https://github.com/hashiprobr/netpixi.git
# !pip install graph-tool-extras

In [2]:
# import graph_tool_extras as gte
import pandas as pd
import csv
import netpixi
import numpy as np

import graph_tool_extras as gte
from graph_tool import draw

/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


ModuleNotFoundError: No module named 'netpixi'

In [ ]:
df = pd.read_csv('csv/imdb_movies.csv')
df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


In [3]:
df['date_x'] = df['date_x'].str.strip().replace('', pd.NA)
df = df.dropna(subset=['date_x'])
df['date_x'] = pd.to_datetime(df['date_x'], format='%m/%d/%Y', errors='coerce')

NameError: name 'df' is not defined

In [4]:
df.dtypes

NameError: name 'df' is not defined

In [5]:
# df_filtered = df[df['date_x'].dt.year > 2020]
df_recente = pd.DataFrame()

# Pega linhas com ano DEPOIS de 2020 e nota MAIOR ou IGUAl a 70.0
for idx, row in df.iterrows():
    ano = row['date_x'].year
    score = row['score']
    # print(score > 70)
    if ano >= 2020 and score > 70.0:
        # print('   Entrou mesmo assim')
        df_recente = pd.concat([df_recente, row.to_frame().T], ignore_index=True)

# Apaga as linhas com valores NaN
df_recente.dropna(subset='names')
df_recente.head()

NameError: name 'df' is not defined

In [6]:
df_recente.head()

# Deleta duplicados (GRRRRRR)
df_recente.drop_duplicates(subset=['names'])

""


In [7]:
df_recente.drop_duplicates(subset='names', inplace=True)
print(df_recente[df_recente.duplicated(subset='names', keep=False)])

Empty DataFrame
Columns: []
Index: []


## Criando o Grafo

In [8]:
# Peguei essa logica da Aula 3, qualquer coisa da uma olhada
g = gte.Graph(directed=False) # igual a gte.Graph(), pois directed é falso por padrão

g.add_vp('color')

NameError: name 'gte' is not defined

In [9]:
filmes = []
for nome in df_recente['names']:
    f = g.add_vertex_by_id(nome)
    f['color'] = 0xfffff0
    filmes.append(f)

KeyError: 'names'

In [ ]:
# pos = gte.layout_bipartite(g, v_tipo)

# graph_draw(g, pos=pos, vertex_text=v_tipo, output_size=(400, 400), output="rede_bipartida.png")

In [ ]:
# Criando o vertice de generos. Antes de fazer isso temos q separar os generos de cada filme, por exemplo: em um filme q tenha Drama, Action como generos, 
# temos q criar uma lista com os dois separados
generos = set()
genero = []
for genre in df_recente['genre']:
    w = g.vertex_by_id(genre)
    # print([genre])
    for ga in genre.split(','):
        g_limpo = ga.strip()
        genero.append(g_limpo)
    generos.update(genero)

lista_generos = sorted(list(generos))

mat_zerada = np.zeros((len(lista_generos), len(lista_generos)))
matriz_gen = pd.DataFrame(mat_zerada, columns=lista_generos, index=lista_generos)

In [ ]:
from pprint import pprint

In [ ]:
# Cria dicionario para salvar ocorrencias
conexoes_gen = {}
for g_prim in generos:
    conexoes_gen[g_prim] = { 'Total' : 0}
    for g_sec in generos:
        if g_sec != g_prim:
            conexoes_gen[g_prim][g_sec] = 0
            # print('Criou')
# pprint(conexoes_gen)
# Ocorrencias de apararecer juntas
for genres in df_recente['genre']:
    for ga in genres.split(','):
        ga = ga.strip()
        # print(f'GA: {ga}')
        for gb in genres.split(','):
            gb = gb.strip()
            if ga != gb:
                # print(f'    GB: {gb}')
                conexoes_gen[ga][gb] += 1
                conexoes_gen[ga]['Total'] += 1

# pprint(conexoes_gen)
percents = {}
percents_mean = {}
# Porcentagem de aparição de cada filme em relação a outro
for genA in conexoes_gen:
    # r_genA = repr(genA)[1:-1]
    # print(f'GA: {genA}')
    # print(f'raw GA: {r_genA}')
    percents[genA] = {}
    mean_p = 0
    for genB in conexoes_gen[genA]:
        # r_genB = repr(genB)[1:-1]
        # print(f'    GB: {genB}')
        # print(f'    raw GB: {r_genB}')
        if genB != 'Total':
            percents[genA][genB] = round(conexoes_gen[genA][genB] * 100 / conexoes_gen[genA]['Total'], 2)
            mean_p += round(conexoes_gen[genA][genB] * 100 / conexoes_gen[genA]['Total'], 2)
    mean_p /= (len(conexoes_gen[genA]) - 1)
    percents_mean[genA] = mean_p
    
# pprint(percents)

# Media de 5.55%, aumentamos para 10% para realizar um corte maior para evitar produções incomuns
print('Media [%] por categoria: ')
pprint(percents_mean)

len(percents)

Media [%] por categoria: 
{'Action': 5.555555555555555,
 'Adventure': 5.554444444444443,
 'Animation': 5.556111111111111,
 'Comedy': 5.555555555555555,
 'Crime': 5.556666666666668,
 'Documentary': 5.554999999999999,
 'Drama': 5.555555555555556,
 'Family': 5.554999999999999,
 'Fantasy': 5.555555555555555,
 'History': 5.556111111111109,
 'Horror': 5.555000000000001,
 'Music': 5.555555555555555,
 'Mystery': 5.555000000000001,
 'Romance': 5.557222222222221,
 'Science Fiction': 5.555,
 'TV Movie': 5.555555555555555,
 'Thriller': 5.553888888888889,
 'War': 5.556111111111112,
 'Western': 5.5572222222222205}


19

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.plot(conexoes_gen['Action'])

In [ ]:
# Popular os dados de generos
for gen in df_recente['genre']:
    # print(gen)
    for ga in gen.split(','):
        # print(ga)
        ga_limpo = ga.strip()
        for gb in gen.split(','):
            gb_limpo = gb.strip()
            matriz_gen.loc[gb_limpo, ga_limpo] = 1

display(matriz_gen)

for gen_a in lista_generos:
    for gen_b in lista_generos:
        if gen_b != gen_a and matriz_gen.loc[gen_b, gen_a] == 1:
            # Adiciona a arresta se foram localizados juntos
            # print(f'Para {gen_a} e {gen_b}, achamos relacao!')
            if g.vertex_by_id(gen_a) is None:
                v = g.add_vertex_by_id(gen_a)
                v['color'] = 0xff0000
                # v_color[v] = "red"
                
            if g.vertex_by_id(gen_b) is None:
                v = g.add_vertex_by_id(gen_b)
                v['color'] = 0xff0000
                # v_color[v] = "red"
                
            g.add_edge_by_ids(gen_a, gen_b)

            # Para nao repetir criacao de aresta
            matriz_gen.loc[gen_b, gen_a] = 0
            matriz_gen.loc[gen_a, gen_b] = 0

            df.sum(axis=1, numeric_only=True)



,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
Action,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Adventure,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
Animation,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Comedy,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
Crime,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
Documentary,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
Drama,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Family,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
Fantasy,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
History,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0


In [ ]:
genero = []

for idx, row in df_recente.iterrows():
    filme = row['names']
    for ga in row['genre'].split(','):
        g_limpo = ga.strip()
        genero.append(g_limpo)
        g.add_edge_by_ids(filme, g_limpo)

In [ ]:
layout = draw.sfdp_layout(g)

gte.move(g, layout)

In [ ]:
print("Número de vértices:", g.num_vertices())
print("Número de arestas:", g.num_edges())

Número de vértices: 707
Número de arestas: 3760


In [ ]:
gte.save(g, 'imdb.net.gz')

r = netpixi.render('imdb.net.gz');
# r.vertex_scale('genre', 1, 5)

In [ ]:
r.vertex_default(
    size=20,         # tamanho            (padrão: 16)
    color=0xff0000,  # cor                (padrão: 0xffffff)
    bwidth=2,        # espessura da borda (padrão: 2)
    bcolor=0x000000, # cor da borda       (padrão: 0x3f3f3f)
)

r.edge_default(
    width=2,        # espessura         (padrão: 2)
    color=0x7777ff, # cor               (padrão: 0xffffff)
    curve1=0,       # curvatura inicial (padrão: 0)
    curve2=0,       # curvatura final   (padrão: 0)nalogamente, para atribuir características visuais às arestas, podemos chamanalogamente, para atribuir características visuais às arestas, podemos chama
)


## Degree (Híbrida)

In [ ]:
g_deg_hibrid = g

c = gte.total_degree(g_deg_hibrid)
g_deg_hibrid.add_vp('degree', c)
gte.save(g_deg_hibrid, 'degree_hib.net.gz')

In [ ]:
r_hib = netpixi.render('degree_hib.net.gz')

In [ ]:
r_hib.vertex_scale('degree', 10, 35)

r_hib.vertex_default(
    size=20,         # tamanho            (padrão: 16)
    color=0xff0000,  # cor                (padrão: 0xffffff)
    bwidth=2,        # espessura da borda (padrão: 2)
    bcolor=0x000000, # cor da borda       (padrão: 0x3f3f3f)
)

r_hib.edge_default(
    width=2,        # espessura         (padrão: 2)
    color=0x7777ff, # cor               (padrão: 0xffffff)
    curve1=0,       # curvatura inicial (padrão: 0)
    curve2=0,       # curvatura final   (padrão: 0)nalogamente, para atribuir características visuais às arestas, podemos chamanalogamente, para atribuir características visuais às arestas, podemos chama
)

In [ ]:
g_ego = g
c = gte.burt_constraint(g_ego)
g_ego.add_vp('burt_constraint', c)
gte.save(g_ego, 'structural_holes.net.gz')

In [ ]:
r_ego = netpixi.render('structural_holes.net.gz')

In [ ]:
r_ego.vertex_scale('burt_constraint', 10, 30)

In [ ]:
r_ego.vertex_default(
    size=20,         # tamanho            (padrão: 16)
    color=0xff0000,  # cor                (padrão: 0xffffff)
    bwidth=2,        # espessura da borda (padrão: 2)
    bcolor=0x000000, # cor da borda       (padrão: 0x3f3f3f)
)

r_ego.edge_default(
    width=2,        # espessura         (padrão: 2)
    color=0x7777ff, # cor               (padrão: 0xffffff)
    curve1=0,       # curvatura inicial (padrão: 0)
    curve2=0,       # curvatura final   (padrão: 0)nalogamente, para atribuir características visuais às arestas, podemos chamanalogamente, para atribuir características visuais às arestas, podemos chama
)

# Modelo Baseline - Variaveis de Controle

In [ ]:
r_hib = netpixi.render('degree_hib.net.gz')
r_hib.head()

NameError: name 'netpixi' is not defined